<a href="https://colab.research.google.com/github/udaysharma1501/mlops_proj1/blob/main/pipeline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)

⠧ Installing integrations...


{'status': 'ok', 'restart': True}

In [1]:
ngrok_authtoken = "2sPSTRBlZN2EokGq13FMrlrvITr_SmdhSuRZFureCs8MGyuY"

In [2]:
!rm -rf .zen
!zenml init

⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠼ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For

In [3]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  # Import scaler
from sklearn.metrics import classification_report, confusion_matrix  # Import metrics

def train_test() -> None:
    """Train and test a Scikit-learn SVC classifier on digits with improvements"""

    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))

    # Scale the data
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=True  # Shuffle is True
    )

    model = SVC(gamma='scale')  # 'scale' sets gamma automatically based on data
    model.fit(X_train, y_train)

    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc*100}%")

    # More detailed evaluation
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

train_test()

Test accuracy: 98.61111111111111%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58
           1       1.00      1.00      1.00        36
           2       0.96      0.96      0.96        23
           3       0.97      0.97      0.97        34
           4       1.00      1.00      1.00        32
           5       0.97      1.00      0.98        32
           6       1.00      1.00      1.00        32
           7       0.95      1.00      0.97        36
           8       1.00      0.97      0.99        35
           9       1.00      0.95      0.98        42

    accuracy                           0.99       360
   macro avg       0.98      0.99      0.98       360
weighted avg       0.99      0.99      0.99       360

[[58  0  0  0  0  0  0  0  0  0]
 [ 0 36  0  0  0  0  0  0  0  0]
 [ 0  0 22  0  0  0  0  1  0  0]
 [ 0  0  1 33  0  0  0  0  0  0]
 [ 0  0  0  0 32  0  0  0  0  0]
 [ 0  0  0  0  0 32  0  0  0  0]
 [ 0  0  

In [4]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """Load the digits dataset as numpy arrays"""
    digits = load_digits()

    data = digits.images.reshape((len(digits.images), -1))

    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test, y_train, y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train and return a Scikit-learn SVC classifier"""
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    return model


@step
def evaluator(
    model: ClassifierMixin,
    X_test: np.ndarray,
    y_test: np.ndarray,
) -> float:
    """Calculate classification accuracy"""
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc*100}%")
    return test_acc

In [5]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  """Links all the steps together in a pipeline"""
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train=X_train, y_train=y_train)
  evaluator(X_test=X_test, y_test=y_test, model=model)

In [6]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Using cached version of step importer.
Using cached version of step svc_trainer.
Using cached version of step evaluator.
All steps of the pipeline run were cached.


In [7]:
!ngrok config add-authtoken "2sPSTRBlZN2EokGq13FMrlrvITr_SmdhSuRZFureCs8MGyuY"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
%pip install pyngrok

In [9]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    print(f"Dashboard accessible at {public_url}")
    !zenml up --blocking --port {port}

  else:
    print(f"Dashboard accessible at http://localhost:{port}")
    !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-f0e9c5a6-339f-440e-88cf-ff8f8e318335


Opening tunnel named: http-8237-f0e9c5a6-339f-440e-88cf-ff8f8e318335


INFO:pyngrok.process.ngrok:t=2025-02-01T05:09:12+0000 lvl=info msg="no configuration paths supplied"


t=2025-02-01T05:09:12+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2025-02-01T05:09:12+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2025-02-01T05:10:51+0000 lvl=info msg="join connections" obj=join id=544cfcd107b6 l=127.0.0.1:8237 r=112.196.126.3:50011
INFO:     112.196.126.3:0 - "GET /api/v1/pipeline_deployments/6ebc2e31-bb21-4263-94f3-3555fb590cd8 HTTP/1.1" 200 OK
An error occurred while processing the request
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/anyio/streams/memory.py", line 98, in receive
    return self.receive_nowait()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/streams/memory.py", line 93, in receive_nowait
    raise WouldBlock
anyio.WouldBlock

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/base.py", line 159, in call_next
    message = await recv_stream.receive()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/streams/memory.py", line 118, in re

INFO:pyngrok.process.ngrok:t=2025-02-01T05:10:55+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2025-02-01T05:10:55+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:pyngrok.process.ngrok:t=2025-02-01T05:10:55+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


t=2025-02-01T05:10:55+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:     Finished server process [11902]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "uvloop/loop.pyx", line 1512, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1505, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1379, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 557, in uvloop.loop.Loop._run
  File "uvloop/loop.pyx", line 476, in uvloop.loop.Loop._on_idle
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.pyx", line 63, in uvloop.loop.Handle._run
  File "/usr/local/lib/python3.11/dist-p